# AES PYTHON

In [5]:
import os , base64
import cryptography
from cryptography.fernet import Fernet as f
import pyfiglet as fb
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import threading
import time
from math import ceil
from termcolor import colored as c

In [7]:

def password(passwd):
    
    password = passwd.encode() # Convert the password string to type bytes using UTF-* encoding
    salt = os.urandom(16)  # used to salt the password for enhanced security
    kdf = PBKDF2HMAC(      #creating a key derivation function object
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
            )
    k = base64.urlsafe_b64encode(kdf.derive(password))
    return k

In [8]:
def encrypt(key,file):
    try:
    
        with open(file,"rb") as fname:
            data = fname.read()
            
        #Extract file name and extension    
        fl,ext = os.path.splitext(file)
        
        #Create a Fernet key from the provided key
        fkey = f(key)
        
        #Encrypt the file data using Fernet
        enc = fkey.encrypt(data)
        
        #Write the encrypted data to new created file with a .en extension
        with open(str(fl[0:])+ext+'.en','wb') as encfile:
            encfile.write(enc)
            
        #Remove the original unencrypted file
        os.remove(file)
        
        #Print the results
        print("                      ")
        print(f"File encrypted sucessfully ")
        print("                               ") 
        print(f"{file} ==> {str(fl[0:])+ext+'.en'}")
        
    except Exception as e:
        print("                      ")
        print("Something went wrong",e)


In [9]:
def decrypt(key,file):
    try:
        
        with open(file, "rb") as fname:
            data = fname.read()
            
        #Create a Fernt key from the provided key
        fkey = f(key)
        
        ##Extract file name and extension
        fl,ext = os.path.splitext(file)
        
        #Decrypt data using Fernet
        dec = fkey.decrypt(data)
        
        #Write the decrypted data to a new file with the original file name
        with open(str(fl[0:]), 'wb') as decfile:
            decfile.write(dec)
            
        #Remove the original encrypted file
        os.remove(file)
        print("                      ")
        print(f"File decrypted sucessfully")
        print("                             ")
        print(f"{file} ==> {fl[0:]}")
    
    except:
        print("                      ")
        print("Something went wrong ")

In [ ]:
def main():
    
    while True:
        
        #Initial message to print on the screen
        c = input("Enter your choice encrypt or decrypt (e/d/q) :").lower()
        listOfFiles = list()
       
        if c =='e':
            print("                      ")
            file_input = input("Enter full path of Targeted file or Dir :")
            print(" ")
            
            #Checks if the path exists
            if os.path.exists(file_input): 
                if file_input !="":    #Checks if a non-empty path is provided
                    passwd = input("Enter your password to encrypt the data :")
                    start = time.time()
                    if os.path.isfile(file_input)==False: #Checks if the input is a directory
                        for (dirpath, dirnames, filenames) in os.walk(file_input):
                            listOfFiles += [os.path.join(dirpath, file) for file in filenames]
                        for l in listOfFiles:    
                            x= threading.Thread(target=encrypt,args=(password(passwd),l)) #Creates a thread for each file 
                            x.start()
                            x.join()
                    else :
                        encrypt(password(passwd),file_input)
                    print('')
                    print("Time taken to encrypt :",ceil(time.time()-start),'seconds')
                else:
                    print("Enter full path")
            else:
                print("Path Provided does not exist")

        elif c =='d':
            print("                      ")
            file_input = input("Enter full path of Targeted file or Dir :")
            if os.path.exists(file_input):
                print(" ")
                
                if file_input !="":  #Checks if a non-empty path is provided
                    passwd = input("Enter your password to decrypt the data :")
                    start = time.time()
                    if os.path.isfile(file_input)==False: #Checks if the input is a directory
                        for (dirpath, dirnames, filenames) in os.walk(file_input):
                            listOfFiles += [os.path.join(dirpath, file) for file in filenames]
                        for l in listOfFiles:    
                            x = threading.Thread(target=decrypt,args=(password(passwd),l))
                            x.start()
                            x.join()
                    else:
                        decrypt(password(passwd),file_input)
                    print('')
                    print("Time taken to decrypt :",ceil(time.time()-start),'seconds')
                else:
                    print("Enter full path")
            else:
                print("Path Provided does not exist")


        elif c =="q" or c =='quit':
            break
        else:
            print(" ")
            print("Please enter a valid Choice !!")

In [ ]:
if __name__=="__main__": #Execute the progran
    main()